# Merging of large dataframes

# UNIT 1

In [1]:
import pymysql
from yaml import load
from datetime import datetime

from merging import *

ModuleNotFoundError: No module named 'pymysql'

In [ ]:
with open('config.yaml', 'r') as f:
    config = load( f )

In [ ]:
connection = pymysql.connect(
    host = config['mysql']['key1']['dbhost'],
    port = comfig['mysql']['key1']['dbport'],
    user = config['mysql']['key1']['dbuser'],
    password = config['mysql']['key1']['dbpass'],
    db = 'conversion',
    cursorclass = pymysql.cursors.SSDictCursor
)

In [ ]:
queryOrdersTech = "Select dateTime, type, region, source, medium, orders as value FROM {}.{};"

In [ ]:
ordersTechData = statusData('conversion', 'orders', connection, queryOrdersTech)

In [ ]:
%%time
i=0

with open('joined_tables.txt', 'w') as f:
    f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('dateTime', 'type', 'region', 'source', 'medium', 'visits', 'orders' ))
    
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM conversion.visits where dateTime >= '2017-06-01'")
        
        for line in cursor:
            
            orders = searchForLine( line['dateTime'], line['type'], line['region'], line['source'], line['medium'], ordersTechData )
            f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(line['dateTime'], line['type'], line['region'], line['source'], line['medium'], line['visits'], orders))
            
            i += 1
            if i % 10000 == 0:
                print( i )

# UNIT 2

In [14]:
head = True
orders_dict = {}
with open('orders_by_source.txt', 'r') as f:
    for line in f:
        if head:
            head = False
        else:
            line = line.strip().split('\t')
            source = line[0]
            orders_count = int(line[1])
            orders_dict[source] = orders_count

In [15]:
print(orders_dict['promo'])

68


In [8]:
# Описание кейса для объединения таблиц
# На данном занятии мы будем использовать данные о покупках из файла,
# сгруппированных по источнику, каналу и дате, а также аналогичный файл по визитам на сайт, 
# часть из которых привела к покупке.

# Файл с визитами по источникам (“visits_by_source.txt”)(в первом столбце стоит источник трафика, во втором — сумма визитов)
# Файл с покупками по источникам (“orders_by_source.txt”)(в первом столбце стоит источник трафика, 
# во втором — количество покупок, в третьем — суммарная стоимость покупок)

# Наша задача — посчитать конверсию из визитов в покупки в разрезе источников трафика.
# Т. е. для каждой пары значений source и medium нам нужно взять количество визитов и покупок и совместить их в одной таблице.

# Файл visits_by_source.txt может быть любого размера (мы обрабатываем его построчно). 
# Представим, что это выгрузка из базы данных на 50 Гб или поток данных в виде транзакций из базы данных.
# То есть мы считаем файл условно «бесконечным».

# Файл orders_by_source.txt помещается в оперативной памяти компьютера. 
# Однако в памяти мы можем обрабатывать довольно большие файлы.
# Например, файл в 1 гигабайт свободно помещается в памяти даже не особо мощного ноутбука.
# То есть файл большой, но помещается в оперативную память.

# Наша задача будет состоять в следующем: нужно написать функцию, на вход которой
# будем подавать очередную строчку из файла visits_by_source.txt. 
# В ответ должны получать количество покупок из файла orders_by_source.txt, 
# которое соответствует этой строчке. Т. е. в самом простом случае 
# в файле orders_by_source.txt надо найти строчку с таким же источником и вернуть значение из второго столбца

def searchForLine( source, orders_dict ):
    """
        Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.
    Возвращает количество покупок, соответствующих источнику source.
        Если источник не найден, но возвращает 0.
    Например:
        >> searchForLine('burgerclub')
        10
        >> searchForLine('source_123')
        0
    """
    # Для получения определенного значения по названию источника отлично подойдет словарь.
    # Можно было, например, поместить наши пары значений в набор листов и пробегать все его значения. 
    # Но это будет очень долго в случае больших файлов. К тому же если источника в файле покупок нет,
    # то "проход" по этому листу будет впустую.
    
    if source in orders_dict:
        return orders_dict[source]
    

In [10]:
print(searchForLine('burgerclub', orders_dict))

10


In [17]:
with open('joined_by_source.txt', 'w') as f_joined:
    with open('visits_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            visits = int(line[1])
            orders = searchForLine(source, orders_dict)
            f_joined.write('{}\t{}\t{}\t{}\n'.format(source, visits, orders, orders/visits))

In [24]:
conv_sum = 0
conv_count = 0
with open('joined_by_source.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
        conv_sum += float(line[3])
        conv_count += 1
print('Average coversion is {:.3f}'.format(conv_sum/conv_count))

Average coversion is 0.007


# UNIT 3

In [ ]:
# Построение вложенного словаря

# Поиск по одному столбцу — ситуация не самая часто встречающаяся. 
# Обычно столбцов гораздо больше. Да и файлы, с которыми приходится работать, на практике гораздо сложнее.

# Для удобства представим более реальную таблицу с покупками. Например, на 4 миллиона строк и 500 Мб на диске.
# Если необходимо найти в этой таблице нужную комбинацию source и medium,
# надо учитывать, что эта пара может встретиться в любой из 4 миллионов строк. 
# Также может быть ситуация, что для этой комбинации source и medium не было ни одной покупки.

# Для ускорения нашего поиска давайте заменим таблицу из столбцов source, medium и количества покупок вложенным словарем.

# Что дает такая структура?
# Теперь, чтобы найти нужную комбинацию значений source и medium, нужно сделать два простых шага:

# 1. Из 2000 значений source определить, есть ли среди них нужный нам. Если его не оказалось, то сразу возвращаем 0 покупок.

# 2. Если значение source есть в первичных ключах, то ищем значение medium в списке из вторичных ключей, 
# которые оказались у источника source (кстати, их может быть всего несколько).
# Логика такая же: если ключ найден, возвращаем значение покупок для первичного ключа source и вторичного medium. 
# Если не найден, то возвращаем 0.

# Итого для поиска среди 4 000 000 строк нам нужно сначала проверить наличие очередного значения в списке ключей из source,
# а затем — в списке из medium. Это намного быстрее, чем любой последовательный мониторинг этих 4 миллионов строк.



In [ ]:
# Метод setdefault
# Можно переписать этот код с помощью метода setdefault. 

# orders_dict = {}
# if 'google' in orders_dict:
    # orders_dict['google'] += 1
# else:
    # orders_dict['google'] = 1
# print(orders_dict)

#>> {'google': 1}

# Этот метод проверяет есть ли в словаре указанный ключ 'google'. 
# Если есть, то оставляет соответствующее значение ключа прежним.
# Если ключа не оказалось, то подставляет указанное нами значение (в примере это значение 0). 
# Тем самым после применения метода setdefault можно смело использовать прибавление 1 к ключу 'google'
# независимо от того, был ли этот ключ в словаре раньше

# orders_dict = {}
# orders_dict.setdefault('google', 0)
# orders_dict['google'] += 1

In [27]:
orders_dict = {}
with open('orders_by_source_and_medium.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
        source = line[0]
        medium = line[1]
        orders_count = int(line[2])
        orders_dict.setdefault(source, {})
        orders_dict[source].setdefault(medium, 0)
        orders_dict[source][medium] = orders_count

In [28]:
orders_dict['google']

{'brand': 6, 'sem': 56, 'seo': 15}

In [37]:
def searchForLine( source, medium, orders_dict):
    """
        Функция по названию источника source и канала medium ищет соответствующую строку в словаре, составленному из данных файла orders_by_source.txt.
    Возвращает количество покупок, соответствующих источнику source и каналу medium.
        Если источник не найден, но возвращает 0.
    Например:
        >> searchForLine('google', 'seo', orders_dict)
        15
        >> searchForLine('google_123', 'seo', orders_dict)
        0
    """
    if source in orders_dict:
        if medium in orders_dict[source]:
            return orders_dict[source][medium]
    return 0

In [38]:
searchForLine('google', 'sem', orders_dict)

56

In [40]:
with open('joined_by_source_medium', 'w') as f_joined:
    with open('visits_by_source_and_medium.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            medium = line[1]
            visits = int(line[2])
            orders = searchForLine(source, medium, orders_dict)
            f_joined.write('{}\t{}\t{}\t{}\t{:.3f}\n'.format(source,medium, visits, orders, orders/visits))

In [ ]:
# TEST

# 5. Дан словарь:
# total_costs = { 'google': 1319, 'yandex': 1818, 'promo': 1181 }
# Необходимо вывести названия источников, отсортированных по алфавиту в обратном порядке
# sorted( total_costs.keys(), reverse = True )

# 7. Как нам перевести следующую дату в тип datetime: 
# date_string = '23.02.2017 15:20'
# datetime.strptime( date_string, '%d.%m.%Y %H:%M' )

# EXTRA UNIT

In [ ]:
# ПОНЯТИЕ РЕКУРСИИ

# Вы наверняка заметили, что в прошлом шаге при учете столбца medium наш код усложнился.
# Приходилось каждый раз проверять наличие новых ключей или использовать setdefault для всех уровней.
# В реальных задачах столбцов будет больше. Например, в некоторых задачах приходится объединять таблицы
# по 5 и более столбцам. Если обходиться проверками или методом setdefault на всех возможных уровнях словарей,
# то наш код будет быстро разрастаться или станет медленным, т. к. заранее сложно предсказать структуру
# вложенных уровней словаря orders_dict.

# В текущем модуле под рекурсией мы будем понимать алгоритм,
# в котором функция может вызывать сама себя с изменяющимся аргументом.

In [ ]:
def decrease_and_print_i( i ):
    if i > 1:
        print(i)
        return decrease_and_print_i( i - 1 )
    else:
        return 1

In [41]:
# Рекурсия и словари

# Все прошлые примеры можно было без труда решить с помощью циклов. Давайте сделаем пример, 
# в котором использование рекурсии позволит нам перевести лист
# >> line = ['2016-10-01', 'burgerclub', 'cpa-partners', 1]
# в словарь
# >> dict2fill = {'2016-10-01': {'burgerclub': {'cpa-partners': 1}}}

In [42]:
line = ['2016-10-01', 'burgerclub', 'cpa-partners', 1]

In [47]:
def fillLevels( lineRemainder ):
    """
    На вход функция принимает часть листа line.
    Берет его первый элемент lineRemainder[0] и вызывает себя с остатком листа line, т. е. с элементами lineRemainder[1:].
    Так продолжаем до тех пор, пока в lineRemainder не останется один элемент
    
    Пример
    >> fillLevels( ['2016-10-01', 'burgerclub', 'cpa-partners', 1] )
    {'2016-10-01': {'burgerclub': {'cpa-partners': 1}}}

    """
    # словарь, который будем пошагово заполнять
    dict2fill = {}
    if len(lineRemainder) > 1:
        dict2fill[lineRemainder[0]] = fillLevels(lineRemainder[1:])
    else:
        return lineRemainder[0]
    return dict2fill
    

In [48]:
fillLevels(line)

{'2016-10-01': {'burgerclub': {'cpa-partners': 1}}}

In [49]:
line = [x for x in range(100)]

In [50]:
fillLevels(line)

{0: {1: {2: {3: {4: {5: {6: {7: {8: {9: {10: {11: {12: {13: {14: {15: {16: {17: {18: {19: {20: {21: {22: {23: {24: {25: {26: {27: {28: {29: {30: {31: {32: {33: {34: {35: {36: {37: {38: {39: {40: {41: {42: {43: {44: {45: {46: {47: {48: {49: {50: {51: {52: {53: {54: {55: {56: {57: {58: {59: {60: {61: {62: {63: {64: {65: {66: {67: {68: {69: {70: {71: {72: {73: {74: {75: {76: {77: {78: {79: {80: {81: {82: {83: {84: {85: {86: {87: {88: {89: {90: {91: {92: {93: {94: {95: {96: {97: {98: 99}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}

In [51]:
def checkLevels( levelDict, level, line ):
    if line[level] in levelDict:
        checkLevels(levelDict[ line[level] ], level+1, line)
        return levelDict
    else:
        levelDict[ line[level] ] = fillLevels( line[ level+1:] )
        return levelDict

In [52]:
data = [

['2016-10-01', 'google', 'sem', 5],

['2016-10-01', 'google', 'seo', 1],

['2016-10-01', 'newsletter', 'email', 1]

]

In [53]:
data_dict = {}
for line in data:
    data_dict = checkLevels( data_dict, 0, line)
print(data_dict)

{'2016-10-01': {'google': {'sem': 5, 'seo': 1}, 'newsletter': {'email': 1}}}


In [54]:
data = [

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_1', 'Хабаровск', 114],

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_2', 'Владивосток', 536],

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_1', 'Магадан', 436]

]

In [55]:
data_dict = {}
for line in data:
    data_dict = checkLevels( data_dict, 0, line)
print(data_dict)

{'2018-01-01': {'google': {'cpc': {'ДФО': {'кампания_1': {'Хабаровск': 114, 'Магадан': 436}, 'кампания_2': {'Владивосток': 536}}}}}}


In [62]:
len(data_dict['2018-01-01']['google']['cpc']['ДФО'])

2

In [72]:
def findLineValue( finalDict, line ):
    if len( line ) > 1:
        if line[0] in finalDict:
            return findLineValue( finalDict[line[0]], line[1:])
        else:
            return 0
    else:
        if line[0] in finalDict:
            return finalDict[ line[0]]
        else:
            return 0

In [69]:
data = [

['2016-10-01', 'google', 'sem', 5],

['2016-10-01', 'google', 'seo', 1],

['2016-10-01', 'newsletter', 'email', 1]

]


In [70]:
data_dict = {}
for line in data:
    data_dict = checkLevels( data_dict, 0, line)
print(data_dict)

{'2016-10-01': {'google': {'sem': 5, 'seo': 1}, 'newsletter': {'email': 1}}}


In [73]:
findLineValue( data_dict, ['2016-10-01', 'google', 'sem'] )

5

In [74]:
findLineValue( data_dict, ['2018-01-01', 'google', 'sem'] )

0

In [75]:
data = [

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_1', 'Хабаровск', 114],

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_2', 'Владивосток', 536],

['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_1', 'Магадан', 436]

]

In [76]:
data_dict = {}
for line in data:
    data_dict = checkLevels( data_dict, 0, line)
print(data_dict)

{'2018-01-01': {'google': {'cpc': {'ДФО': {'кампания_1': {'Хабаровск': 114, 'Магадан': 436}, 'кампания_2': {'Владивосток': 536}}}}}}


In [77]:
visits = 26800
conversion = float(findLineValue( data_dict, ['2018-01-01', 'google', 'cpc', 'ДФО', 'кампания_2', 'Владивосток']))/ visits

In [78]:
conversion

0.02